In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
import os
from langchain_openai import ChatOpenAI  # Notez le changement ici

# Définition de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0xUZ6aBpi1 
# Création du modèle
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Notez que c'est 'model' et non 'model_name' dans les versions récentes
    temperature=0
)

In [7]:
dense_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate responses using Demonstration Ensembling (DENSE) technique based on user questions.

    Demonstration Ensembling (DENSE) is an advanced prompt engineering technique that creates multiple few-shot prompts, each containing distinct subsets of exemplars, and then aggregates their outputs to generate a more accurate final response.

    This technique reduces variance in model outputs and improves accuracy by leveraging diverse perspectives from different sets of examples.

    The DENSE process includes the following steps:
    1. Create multiple few-shot prompts with different example subsets
    2. Generate responses for each prompt separately
    3. Aggregate the responses using appropriate methods (majority vote, weighted combination, etc.)
    4. Construct the final output based on the aggregation results

    Please structure your response in the following format:

    Prompt Set 1: [3-5 diverse examples related to the task]
    Response 1: [Generated solution based on Prompt Set 1]

    Prompt Set 2: [3-5 different examples related to the task]
    Response 2: [Generated solution based on Prompt Set 2]

    Prompt Set 3: [3-5 alternative examples related to the task]
    Response 3: [Generated solution based on Prompt Set 3]

    Aggregation Analysis:
    - Common elements across responses
    - Unique insights from each response
    - Areas of consensus and disagreement
    - Confidence assessment for different solution aspects

    Final Response:
    [Comprehensive solution that integrates the best elements from all generated responses]

    Example:
    Question: "What are effective strategies for reducing carbon emissions in urban areas?"

    Prompt Set 1:
    Example 1: "How can factories reduce air pollution?" - Implement advanced filtration systems, switch to cleaner energy sources, optimize manufacturing processes to minimize waste.
    Example 2: "What methods exist for lowering vehicle emissions?" - Promote electric vehicles, improve public transportation, implement congestion charges.
    Example 3: "How can power plants become more environmentally friendly?" - Transition to renewable energy sources, implement carbon capture technologies, improve energy efficiency.
    Response 1: Urban carbon emission reduction strategies should focus on transportation electrification, industrial emission controls, and renewable energy implementation for municipal power needs.

    Prompt Set 2:
    Example A: "How can cities reduce waste?" - Implement comprehensive recycling programs, encourage composting, reduce single-use plastics.
    Example B: "What are methods for improving energy efficiency in buildings?" - Upgrade insulation, install smart climate control systems, use energy-efficient appliances.
    Example C: "How can urban planning reduce environmental impact?" - Create walkable neighborhoods, increase green spaces, implement sustainable water management.
    Response 2: Urban carbon emission reduction requires integrated approaches: building efficiency improvements, urban density planning that reduces transportation needs, and waste management systems that minimize methane from landfills.

    Prompt Set 3:
    Sample I: "How can consumers make eco-friendly choices?" - Choose locally produced goods, reduce meat consumption, minimize unnecessary purchases.
    Sample II: "What policy measures can incentivize green practices?" - Carbon taxes, subsidies for renewable energy, regulations on emissions standards.
    Sample III: "How can technology innovation address climate change?" - Smart grid development, improved battery storage, carbon capture technologies.
    Response 3: Effective urban carbon reduction requires policy frameworks (carbon pricing, building codes), technological solutions (smart grids, EV infrastructure), and behavior change campaigns to engage citizens.

    Aggregation Analysis:
    - Common elements: All responses emphasize transportation, buildings, and policy measures
    - Unique insights: Response 1 focuses on industrial applications, Response 2 highlights urban planning, Response 3 emphasizes citizen engagement
    - Consensus areas: Transportation and building efficiency are universally recognized priorities
    - Varying confidence: Highest confidence in transportation solutions, moderate confidence in behavioral approaches

    Final Response:
    Urban carbon emission reduction requires a multi-faceted approach combining:
    1. Transportation reforms: Electrification of public transit, EV infrastructure development, and designing compact, walkable neighborhoods to reduce vehicle dependence
    2. Building optimization: Implementing strict energy efficiency standards for new construction and retrofitting existing buildings with better insulation and smart energy management systems
    3. Energy transition: Shifting municipal power generation to renewable sources and implementing district heating/cooling where feasible
    4. Policy frameworks: Establishing carbon pricing mechanisms, green building codes, and financial incentives for low-carbon solutions
    5. Citizen engagement: Developing programs to encourage sustainable consumption, waste reduction, and community participation in green initiatives

    This comprehensive approach addresses both the technical and social dimensions of urban carbon emissions while balancing immediate impact with long-term sustainability.
    """),
    ("human", "{question}"),
    ("assistant", """
    I'll analyze your question using the Demonstration Ensembling (DENSE) technique to provide a comprehensive and accurate response.

    Prompt Set 1:
    [Select 3-5 diverse, relevant examples related to the user's question. These examples should cover different aspects of the problem domain.]
    Example 1: [Question-answer pair covering first aspect]
    Example 2: [Question-answer pair covering second aspect]
    Example 3: [Question-answer pair covering third aspect]
    Response 1: [Generate a detailed response based solely on the examples in Prompt Set 1]

    Prompt Set 2:
    [Select 3-5 different examples that approach the problem from alternative angles]
    Example A: [Question-answer pair with alternative perspective 1]
    Example B: [Question-answer pair with alternative perspective 2]
    Example C: [Question-answer pair with alternative perspective 3]
    Response 2: [Generate a detailed response based solely on the examples in Prompt Set 2]

    Prompt Set 3:
    [Select 3-5 additional examples that provide yet another perspective on the problem]
    Sample I: [Question-answer pair with additional perspective 1]
    Sample II: [Question-answer pair with additional perspective 2]
    Sample III: [Question-answer pair with additional perspective 3]
    Response 3: [Generate a detailed response based solely on the examples in Prompt Set 3]

    Aggregation Analysis:
    - Common elements across responses: [Identify concepts, approaches, or solutions that appear in multiple responses]
    - Unique insights from each response: [Highlight valuable unique contributions from each response]
    - Areas of consensus and disagreement: [Note where the responses align and where they differ]
    - Confidence assessment: [Assess confidence levels for different aspects of the solution]

    Final Response:
    [Provide a comprehensive, integrated solution that combines the strongest elements from all three responses. The final response should be more robust, accurate, and complete than any individual response.]
    """)
])

# Demonstration Ensembling (DENSE)

Demonstration Ensembling, also known by the acronym DENSE and introduced by Khalifa et al. (2023), is an innovative method that enhances the performance of large language models (LLMs) by leveraging the diversity of learning examples.

## How Does Demonstration Ensembling Work?

Imagine you're preparing a student for a complex exam. Instead of offering them a single set of practice exercises, you could be more effective by presenting several different sets of examples, then combining the best problem-solving strategies from each set.

Demonstration Ensembling follows this logic in three key steps:

1. **Creation of multiple prompts**: Several few-shot prompts are generated, each containing a distinct subset of examples relevant to the task.

2. **Generation of independent responses**: The model produces a response for each prompt, with each response being influenced by the specific examples in its corresponding prompt.

3. **Intelligent aggregation**: The different responses are analyzed and combined to produce a final response that integrates the strengths of each individual response.

## For which types of questions is DENSE particularly suitable?

The DENSE technique is particularly effective for the following types of questions:

### Complex Multidimensional Analyses
- Market analysis problems requiring different sectoral perspectives
- Public policy questions involving multiple domains of expertise
- Interdisciplinary scientific analyses requiring various methodological approaches
- Strategic assessments needing integration of multiple factors

### Creative Problem Solving
- Design of innovative solutions benefiting from multiple approaches
- Development of communication strategies for different audiences
- Elaboration of action plans requiring varied perspectives
- Resolution of technological challenges involving diverse considerations

### Decision Making
- Risk assessments requiring different analytical perspectives
- Strategic choices benefiting from multiple decision frameworks
- Ethical questions involving various frames of reference
- Trend predictions integrating different extrapolation approaches

### Concrete Examples of Questions

1. "What would be the most effective strategies to reduce carbon emissions in urban areas?"
   - DENSE would use one set of examples on transportation, another on building energy efficiency, and a third on consumer behaviors

2. "How could employee engagement be improved in a company undergoing digital transformation?"
   - DENSE would leverage examples on training, others on reward systems, and still others on internal communication

3. "What factors should be considered when developing a new food product for international markets?"
   - DENSE would combine examples on cultural preferences, regulations, and logistical considerations

4. "How could we design an educational system that better prepares students for the skills required in the 21st century?"
   - DENSE would integrate examples on innovative pedagogies, socio-emotional skills development, and technological integration

5. "What approaches might be most effective for improving the resilience of global supply chains?"
   - DENSE would use examples on supplier diversification, traceability technologies, and storage strategies

Demonstration Ensembling is particularly effective for these types of questions because:

- It reduces the variance and biases inherent in a single set of examples
- It improves the robustness of responses by integrating diverse perspectives
- It effectively manages uncertainty by aggregating multiple approaches
- It produces more complete and nuanced responses
- It mimics the collective decision-making process of expert groups who compare their analyses before formulating a conclusion

In [8]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the DENSE prompt and extract the final answer
def generate_dense_prompt_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = dense_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final answer from the Final Response section
    final_answer_pattern = r'Final Response:([\s\S]*?)(?=$|Prompt Set \d:)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Final Response:\s*([\s\S]*?)$'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('dense_prompt_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL ANSWER (AGGREGATED RESPONSE) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_dense_prompt", generate_dense_prompt_node)

# Graph configuration
graph_builder.set_entry_point("generate_dense_prompt")
graph_builder.add_edge("generate_dense_prompt", END)
graph = graph_builder.compile()

# Usage example
inputs = {"messages": [HumanMessage(content="What are the most effective strategies for reducing carbon emissions in urban areas while maintaining economic growth?")],
          "result_df": None}  # Initialize result_df to None


# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL ANSWER (AGGREGATED RESPONSE):")

# Graph execution - only display Final Response
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_dense_prompt":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'dense_prompt_result.txt')")

QUESTION:
What are the most effective strategies for reducing carbon emissions in urban areas while maintaining economic growth?

FINAL ANSWER (AGGREGATED RESPONSE):
To effectively reduce carbon emissions in urban areas while maintaining economic growth, a multi-faceted approach is essential. Key strategies include:

1. **Enhancing Public Transportation**: Invest in reliable and efficient public transit systems, subsidize fares, and create incentives for using public transport over personal vehicles to reduce reliance on cars.

2. **Promoting Renewable Energy**: Implement tax incentives for solar and wind energy projects, and establish regulations that require new buildings to incorporate renewable energy sources, thus fostering a green economy.

3. **Smart Urban Planning**: Design mixed-use developments that minimize travel distances, increase green spaces, and promote walkability, which can significantly lower emissions while enhancing urban livability.

4. **Engaging Businesses**: E